In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Model Legend

* m5 = AZ
* m7 = EZ
* m9 = EFA
* m10 = EFA-C

## Load Model Evidence

In [2]:
log_dirs = dict()
log_dirs['m5'] = 'log/fabian_sim/20210829_134729_sim_m5/'
log_dirs['m7'] = 'log/fabian_sim/20210829_134725_sim_m7/'
log_dirs['m9'] = 'log/fabian_sim/20210829_134730_sim_m9/'
log_dirs['m10'] = 'log/fabian_sim/20210829_134734_sim_m10/'

model_nums = [5,7, 9, 10]

# load existing results
bf = dict()

for model in model_nums:
    bf['m'+str(model)] = load_obj('log_lklhds', log_dirs['m'+str(model)])



In [3]:
data = load_obj('data', log_dirs['m5'])
data.raw_data


{'random_seed': 0,
 'N': 200,
 'K': 2,
 'J': 6,
 'alpha': array([0., 0., 0., 0., 0., 0.]),
 'beta': array([[1. , 0. ],
        [0.8, 0.3],
        [0.8, 0. ],
        [0. , 1. ],
        [0.3, 0.8],
        [0.3, 0.8]]),
 'sigma_z': array([0.80622577, 0.80622577]),
 'Phi_corr': array([[1. , 0.2],
        [0.2, 1. ]]),
 'Phi_cov': array([[0.65, 0.13],
        [0.13, 0.65]]),
 'Marg_cov': array([[1.    , 0.559 , 0.52  , 0.13  , 0.299 , 0.299 ],
        [0.559 , 1.    , 0.4472, 0.299 , 0.4069, 0.4069],
        [0.52  , 0.4472, 1.    , 0.104 , 0.2392, 0.2392],
        [0.13  , 0.299 , 0.104 , 1.    , 0.559 , 0.559 ],
        [0.299 , 0.4069, 0.2392, 0.559 , 1.    , 0.5369],
        [0.299 , 0.4069, 0.2392, 0.559 , 0.5369, 1.    ]]),
 'Theta': array([[0.35  , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.4631, 0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.584 , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.35  , 0.    , 0.    ],
        [0.    , 0. 

In [4]:
bfdf = pd.DataFrame.from_dict(bf)

bfdf['t']  = np.arange(len(bfdf))
for i in [7, 9, 10]:
    bfdf['m5/m'+str(i)] = bfdf['m5']/bfdf['m'+str(i)]
bfdf.drop(['m5', 'm7', 'm9', 'm10'], axis=1, inplace=True)
bfdf = bfdf.melt(id_vars='t', var_name='model')
bfdf = bfdf[bfdf.t>30]

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [5]:
width = 600
height = 400
c = alt.Chart(bfdf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)

In [6]:
model_evidence = dict()
for m in ['m'+str(i) for i in model_nums]:
    model_evidence[m] = np.exp(logsumexp(bf[m][30:]))

model_evidence

{'m5': 0.20864873062630396,
 'm7': 0.20917967761115294,
 'm9': 0.20727873811851763,
 'm10': 0.19833891740261375}

## Scoring Rules


In [7]:
# load existing results
scrl = dict()
for model in model_nums:
    scrl['m'+str(model)] = load_obj('scoring_rule', log_dirs['m'+str(model)])


In [8]:
scrldf = pd.DataFrame.from_dict(scrl)

scrldf['t']  = np.arange(len(scrldf))
scrldf = scrldf[scrldf.t>31]

for i in [7,9,10]:
    scrldf['m'+str(i)+'-m5'] =(scrldf['m'+str(i)] -  scrldf['m5']).cumsum()
scrldf.drop(['m5', 'm7', 'm10', 'm9'], axis=1, inplace=True)
scrldf = scrldf.melt(id_vars='t', var_name='model')

# scrldf['cval'] = scrldf.value.cumsum()

In [9]:
width = 600
height = 400
c = alt.Chart(scrldf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height)
c

alt.Chart(...)